<a href="https://colab.research.google.com/github/bearpelican/musicautobot/blob/master/notebooks/multitask_transformer/Generate_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!git clone https://github.com/bearpelican/musicautobot.git

In [0]:
import os
os.chdir('musicautobot')

In [0]:
!nvidia-smi

In [0]:
!apt install musescore fluidsynth
!cp /usr/share/sounds/sf2/FluidR3_GM.sf2 ./font.sf2
!pip install torch fastai music21 pebble fluidsynth midi2audio

In [0]:
from musicautobot.numpy_encode import *
from musicautobot.utils.file_processing import process_all, process_file
from musicautobot.config import *
from musicautobot.music_transformer import *
from musicautobot.multitask_transformer import *
from musicautobot.numpy_encode import stream2npenc_parts
from musicautobot.utils.setup_musescore import setup_musescore
setup_musescore()

In [0]:
from midi2audio import FluidSynth
from IPython.display import Audio

In [0]:
# Colab cannot play music directly from music21 - must convert to .wav first
def play_wav(stream):
    out_midi = stream.write('midi')
    out_wav = str(Path(out_midi).with_suffix('.wav'))
    FluidSynth("font.sf2").midi_to_audio(out_midi, out_wav)
    return Audio(out_wav)


# Generate Music with Pretrained Model

### Load Pretrained

In [0]:
# Config
config = multitask_config();

# Location of your midi files
midi_path =  Path('data/midi')

# Location of saved datset
data_path = Path('data/numpy')
data_save_name = 'musicitem_data_save.pkl'

In [0]:
# Data
data = MusicDataBunch.empty(data_path)
vocab = data.vocab

In [0]:
# Pretrained Model

# Download pretrained model if you haven't already
pretrained_url = 'https://ashaw-midi-web-server.s3-us-west-2.amazonaws.com/pretrained/MultitaskSmallKeyC.pth'
# pretrained_url = 'https://ashaw-midi-web-server.s3-us-west-2.amazonaws.com/pretrained/MultitaskSmall.pth'

pretrained_path = data_path/'pretrained'/Path(pretrained_url).name
pretrained_path.parent.mkdir(parents=True, exist_ok=True)
download_url(pretrained_url, dest=pretrained_path)

In [0]:
# Learner
learn = multitask_model_learner(data, pretrained_path=pretrained_path)
# learn.to_fp16();

### Choose existing midi file as a starting point

In [0]:
example_dir = midi_path/'examples'
midi_files = get_files(example_dir, recurse=True, extensions='.mid'); midi_files[:5]

In [0]:
file = midi_files[3]; file

In [0]:
# Encode file 
item = MusicItem.from_file(file, data.vocab)

x = item.to_tensor()
x_pos = item.get_pos_tensor()

In [0]:
item.show()

In [0]:
# item.play()
play_wav(item.stream)

## Generate

MultitaskTransformer trains on 3 separate tasks. 
1. NextWord
2. Mask
3. Sequence to Sequence

Because we train on 3 separate tasks, we can actually generate some really cool note sequences.

1. NextWord/Autocomplete - Take a sequence of notes and predict the next note
 * 1a. Vanilla Language Model predictions - See [MusicTransformer](../music_transformer) project


2. Mask/Remix - Mask certain parts of song and remix those portions.
 * 2a. Note Masking - Mask all the note pitches and create a new sequence with different notes, but same exact rhythm
 * 2b. Duration Masking - Mask the note durations. Generate a new sequence with the same melody, but with a different rhythm


3. Seq2Seq/Translation - Generate melody from chords or vice versa. 
 * 3a. New Melody - Generate a new melody from existing chords
 * 3b. Harmonization - Generate chords to acompany an existing melody

## 1. NextWord/Autocomplete

Trim the song to only a few notes. Model will use these notes a seed and continue the idea

In [0]:
seed_len = 6 # 4 beats = 1 bar
seed = item.trim_to_beat(seed_len)

In [0]:
seed.show()

In [0]:
pred_nw, full = learn.predict_nw(seed, n_words=200)

In [0]:
pred_nw.show()

In [0]:
play_wav(pred_nw.stream)

Add more randomness

In [0]:
pitch_temp = 1.4 # randomness of melody
tempo_temp = 1.0 # randomness or rhythm
top_k = 40
pred_nw_rand, full = learn.predict_nw(seed, temperatures=(pitch_temp, tempo_temp), top_k=top_k, top_p=0.5)
pred_nw_rand.show()

In [0]:
play_wav(pred_nw_rand.stream)

In [0]:
# Convenience function
# out = nw_predict_from_midi(learn, file, seed_len=seed_len, top_k=30, top_p=0.5); out.show()

## 2. Seq2Seq/Translation

Load MultitrackItem.

MultitrackItem keeps track of which notes are part of the melody and which notes are part of the chords.  
This info is needed for translation task

In [0]:
multitrack_item = MultitrackItem.from_file(file, vocab)

In [0]:
melody, chords = multitrack_item.melody, multitrack_item.chords

In [0]:
melody.show()

In [0]:
chords.show()

In [0]:
multitrack_item.play()

In [0]:
play_wav(multitrack_item.stream)

## 2a. Create Melody

Use existing chord progression to generate a new melody

In [0]:
# Use a seed for the melody
partial_melody = melody.trim_to_beat(4)

# Or generate from an empty sequence
empty_melody = MusicItem.empty(vocab, seq_type=SEQType.Melody)

In [0]:
seed_melody = empty_melody; seed_melody.show()

In [0]:
pred_melody = learn.predict_s2s(chords, seed_melody, use_memory=True)
pred_melody.show()

In [0]:
play_wav(pred_melody.stream)

In [0]:
combined = MultitrackItem(pred_melody, chords)
combined.show()

In [0]:
play_wav(combined.stream)

## 2b. Harmonization

Generate chords to accompany an existing melody

In [0]:
# partial_chords = chords.trim_to_beat(3);
# partial_chords.show()

empty_chords = MusicItem.empty(vocab, seq_type=SEQType.Chords); empty_chords.show()

In [0]:
pred_chord = learn.predict_s2s(input_item=melody, target_item=empty_chords)

In [0]:
pred_chord.show()

In [0]:
combined = MultitrackItem(melody, pred_chord)
combined.show()

In [0]:
play_wav(combined.stream)

In [0]:
# Convenience Function

# out = s2s_predict_from_midi(learn, file, seed_len=10); out.show()

## 3. Mask/Remix

### 3a. Remix Notes

Mask all the note pitches. Model will create a new song with the same rhythm

In [0]:
### Mask notes
note_item = item.mask_pitch();

In [0]:
# Mask vs Original
list(zip(note_item.to_text(None)[:20], item.to_text(None)[:20]))

In [0]:
pred_note = learn.predict_mask(note_item, temperatures=(1.4, 1.0))

In [0]:
pred_note.show()

In [0]:
play_wav(pred_note.stream)

### 3b. Remix rhythm

Mask note durations. Same notes, different rhythm

In [0]:
# duration mask
dur_item = item.mask_duration()

In [0]:
# Mask vs Original
list(zip(dur_item.to_text(None)[:10], item.to_text(None)[:10]))

In [0]:
dur_pred = learn.predict_mask(dur_item, temperatures=(0.8,0.8), top_k=40, top_p=0.6)

In [0]:
dur_pred.show()

In [0]:
play_wav(dur_pred.stream)

In [0]:
# Convenience function
# out = mask_predict_from_midi(learn, file, predict_notes=True)